In [1]:
import torch

# vezme se bottleneck vektor a vynásobí se query maticí a dostaneme query vektor, to se dělá zvlášť (ne paralelně)
# pro každý token vypočítat jeho K a V (pro každý modul jiný) - každý modul má jinou K a V matici
# 1 matice, kterou když splitnu dostanu K a V - jako u c_attn(), jen to musíme paralelizovat na N modulů
# tahle matice bude tensor o N dimenzích, 1 dimenze = 1 modul
# 4x16 sloupců budou embeddingy tokenů, musí mít stejnou hloubku jako tensor a budou se násobit 1:1, tím získám K V vektory pro každý modul


In [2]:
batch = 64
num_tokens = 4
num_modules = 16
dim_size = 32

In [3]:
in_t = torch.rand(batch, num_tokens*num_modules, dim_size)
print(in_t.shape)

in_t = in_t.reshape(batch, dim_size, -1, num_modules) # 4 počet tokenů pro modul, 32 dimenze embeddingů, 16 počet modulů
print(in_t.shape)

torch.Size([64, 64, 32])
torch.Size([64, 32, 4, 16])


In [4]:
m = torch.rand(2*dim_size, dim_size, num_modules) # 16 modulů, 2*32 - 2 embedding vektory o velikosti 32, 32 embedding vstupu
m.shape

# tensor A který má velikost 4*2*3 (4 - řádky, 2 - sloupce, 3 - matice)
# s tensorem B který má velikost 2*2*3 (2 - řádky, 2 - sloupce, 3 - matice)

torch.Size([64, 32, 16])

In [5]:
m_param = torch.nn.Parameter(m)
m = m_param

In [6]:
result = torch.einsum('ijk,bjlk->bilk', m, in_t)
print(result.shape)  # Should be (64, 4, 16)

torch.Size([64, 64, 4, 16])


In [7]:
# vypočítat query vektor (bottleneck vynásobíme query maticí 32*32) - bottleneck náhodný vektor o dimenzi 32, query matice taky náhodná
# potom vypočítat attention - skalární součin mezi každým key vektorem (první půlka řádků v každé matici) a query maticí
# tím získám logity, na to softmax
# 2 varianty - softmax v rámci každého modulu, agregace v rámci modulu těch value vektorů
# softmax se vypočítá přes všechno najednou, v rámci všech modulů

In [8]:
bottleneck = torch.rand(batch, 32)
q_matrix = torch.rand(batch, 32, 32)

q_vec = torch.bmm(bottleneck.unsqueeze(1), q_matrix).squeeze(1)

In [9]:
q_vec.shape

torch.Size([64, 32])

In [10]:
m.shape

torch.Size([64, 32, 16])

In [11]:
key_vectors = result[:, :dim_size, :, :]
att = torch.einsum("bijk,bi->bjk", key_vectors, q_vec)
#attention_scores = torch.sum(key_vectors * q_vec_reshaped, dim=1)
#attention_scores.shape

In [12]:
att.shape

torch.Size([64, 4, 16])

In [13]:
# (32, 16)
# flattened_scores = att.reshape(-1)
import torch.nn.functional as F
attention_probs = F.softmax(att.reshape(batch, -1), dim=1)

value_vectors = result[:, dim_size:, :, :].reshape(batch, dim_size, -1)  # (32,64)

aggregated_values = torch.einsum('bij,bj->bi', value_vectors, attention_probs)  # (32, 16)

aggregated_values.shape

torch.Size([64, 32])